<a href="https://colab.research.google.com/github/kategimranova/ego_network_analysis/blob/main/collect_data_from_vk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transliterate

In [ ]:
import requests
import json
from transliterate import translit
import itertools

In [ ]:
'''Function for getting list of friends of a person with a given user_id. In name_info == True, the function will return the list of friends 
with some additional data like name
More info about getting access token: https://dev.vk.com/api/access-token/getting-started
'''
def get_friends_ids(user_id, my_id,access_token, name_info=False):
  if name_info:
    friends_url = 'https://api.vk.com/method/friends.get?user_id={}&fields=nickname&v=5.131&access_token={}'.format(user_id, access_token)
  else:
    friends_url = 'https://api.vk.com/method/friends.get?user_id={}&v=5.131&access_token={}'.format(user_id, access_token)
  response = requests.get(friends_url)
  assert response.status_code == 200, "No response recieved"
  if 'This profile is private' in response.content.decode("utf-8"):
    return []
  json_response = json.loads(response.content)
  friends_list = json_response['response']['items'].copy()
  if user_id != my_id: #Deleting me from friends' list (for correct identifying relationship between friends)
    if my_id in friends_list:
      friends_ids = [friend for friend in friends_list if friend!=my_id]
    else:
      friends_ids = friends_list
  else:
    friends_ids = friends_list
  return friends_ids

In [ ]:
''' Function for getting dict with name info from friends_ids (with name_info == True)'''
def get_dict_with_names(friends_list):
  result = {}
  for friend in friends_list:
    result[friend['id']] = friend['first_name'] + " " + friend['last_name']
  
  return result

In [ ]:
'''Function for identifying whether two users are friends'''
def are_friends(user1, user2, friends_dict):
  friends_of_user1 = friends_dict[user1]
  friends_of_user2 = friends_dict[user2]
  return user1 in friends_of_user2 or user2 in friends_of_user1 #рассматриваем для закрытых аккаунтов

In [ ]:
with open('config.json', 'r') as config_file:
  config = json.load(config_file)

In [ ]:
access_token = config['access_token']
my_id = config['my_id']

In [ ]:
my_friends_ids = get_friends_ids(my_id,my_id, access_token, name_info=False)
my_friends_name_info = get_friends_ids(my_id,my_id, access_token, name_info=True)
my_friends_names_rus = get_dict_with_names(my_friends_name_info)

In [ ]:
'Names of friends written on English language'
my_friends_names_eng = {}
for id, name in my_friends_names_rus.items():
  my_friends_names_eng[id] = translit(name, language_code='ru', reversed=True)

In [ ]:
list(my_friends_names_eng.values())[:15]

In [ ]:
friends_dict = {}
for friend in my_friends_ids:
  if friend not in friends_dict:
    friends_dict[friend] = get_friends_ids(friend,my_id, access_token)

In [ ]:
#generating all possible pairs of friends
friends_pairs = list(itertools.combinations(my_friends_ids,2))

In [ ]:
#getting edges of ego-graph
graph_edges = []
for user1, user2 in friends_pairs:
  if are_friends(user1, user2, friends_dict):
    graph_edges.append((user1, user2))

In [ ]:
def write_data_in_txt_file(data, filename):
  with open(filename, 'w') as f:
    f.write(str(data))
  f.close()

In [ ]:
write_data_in_txt_file(graph_edges, 'edges.txt')

In [ ]:
write_data_in_txt_file(my_friends_ids, 'my_friends_ids.txt')

In [ ]:
write_data_in_txt_file(friends_dict, 'friends_dict.txt')

In [ ]:
write_data_in_txt_file(my_friends_names_eng, 'my_friends_names.txt')